In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from tqdm import tqdm
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
%matplotlib inline
test_path = 'https://media.githubusercontent.com/media/cchopade/gahack2/master/test.csv'
train_path = 'https://media.githubusercontent.com/media/cchopade/gahack2/master/train.csv'
import warnings
warnings.filterwarnings("ignore")

In [130]:
#df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
#df_train.Datetime = pd.to_datetime(df_train.Datetime)

df_train=pd.read_csv(train_path,index_col='Datetime',parse_dates=True)
df_train.drop(labels = 'ID', axis = 1)
df_train.head()

,ID,Item_ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales
Datetime,,,,,,,
2014-01-01,30495_20140101,30495,0,2.0,90,165.123,1
2014-01-01,30375_20140101,30375,0,2.0,307,68.666,5
2014-01-01,30011_20140101,30011,0,3.0,67,253.314,2
2014-01-01,30864_20140101,30864,0,2.0,315,223.122,1
2014-01-01,30780_20140101,30780,1,2.0,132,28.750,1


In [76]:
df_train.columns

Index(['ID', 'Item_ID', 'Category_3', 'Category_2', 'Category_1', 'Price',
       'Number_Of_Sales'],
      dtype='object')

In [84]:
df_train.groupby(['Datetime','Item_ID'])['Price']

In [32]:
df = pd.read_csv(train_path)
df.Datetime = pd.to_datetime(df.Datetime)
df = df.set_index(['Item_ID','Datetime'])

In [33]:
df.head()

,,ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales
Item_ID,Datetime,,,,,,
30495,2014-01-01,30495_20140101,0,2.0,90,165.123,1
30375,2014-01-01,30375_20140101,0,2.0,307,68.666,5
30011,2014-01-01,30011_20140101,0,3.0,67,253.314,2
30864,2014-01-01,30864_20140101,0,2.0,315,223.122,1
30780,2014-01-01,30780_20140101,1,2.0,132,28.750,1


In [34]:
df = df.sort_index(ascending=False)
df.sort_values(['Item_ID','Datetime'],ascending=[False,True],inplace=True)
df.reset_index(inplace=True)
df.head()

,Item_ID,Datetime,ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales
0,31375,2015-05-27,31375_20150527,0,5.0,235,0.882,133
1,31375,2015-05-28,31375_20150528,0,5.0,235,0.787,113
2,31375,2015-05-29,31375_20150529,0,5.0,235,0.942,59
3,31375,2015-05-30,31375_20150530,0,5.0,235,1.049,56
4,31375,2015-05-31,31375_20150531,0,5.0,235,1.381,37


In [35]:
items = list(df.Item_ID.unique())

In [38]:
df['day'] = df["Datetime"].apply(lambda x: x.day)
df['week'] = df["Datetime"].apply(lambda x: x.week)
df['month'] = df["Datetime"].apply(lambda x: x.month)
df['year'] = df["Datetime"].apply(lambda x: x.year)
df.head()

,Item_ID,Datetime,ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales,day,week,month,year
0,31375,2015-05-27,31375_20150527,0,5.0,235,0.882,133,27,22,5,2015
1,31375,2015-05-28,31375_20150528,0,5.0,235,0.787,113,28,22,5,2015
2,31375,2015-05-29,31375_20150529,0,5.0,235,0.942,59,29,22,5,2015
3,31375,2015-05-30,31375_20150530,0,5.0,235,1.049,56,30,22,5,2015
4,31375,2015-05-31,31375_20150531,0,5.0,235,1.381,37,31,22,5,2015


In [40]:
item1 = df[df.Item_ID == 30258]
item1.set_index(item1.Datetime,inplace=True)
item1.drop(labels=['ID'], axis = 1,inplace=True)
item1.head()

/home/cchopade/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Item_ID,Datetime,Category_3,Category_2,Category_1,Price,Number_Of_Sales,day,week,month,year
Datetime,,,,,,,,,,,
2014-01-01,30258,2014-01-01,1,5.0,379,0.500,43,1,1,1,2014
2014-01-02,30258,2014-01-02,1,5.0,379,0.487,59,2,1,1,2014
2014-01-03,30258,2014-01-03,1,5.0,379,0.509,36,3,1,1,2014
2014-01-04,30258,2014-01-04,1,5.0,379,0.572,65,4,1,1,2014
2014-01-05,30258,2014-01-05,1,5.0,379,0.571,55,5,1,1,2014


In [41]:
item1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 885 entries, 2014-01-01 to 2016-06-03
Data columns (total 11 columns):
Item_ID            885 non-null int64
Datetime           885 non-null datetime64[ns]
Category_3         885 non-null int64
Category_2         885 non-null float64
Category_1         885 non-null int64
Price              885 non-null float64
Number_Of_Sales    885 non-null int64
day                885 non-null int64
week               885 non-null int64
month              885 non-null int64
year               885 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(8)
memory usage: 83.0 KB


In [66]:
item1.iloc[:90,:]['Price']

Datetime
2014-01-01    0.500
2014-01-02    0.487
2014-01-03    0.509
2014-01-04    0.572
2014-01-05    0.571
2014-01-06    0.565
2014-01-07    0.565
2014-01-08    0.597
2014-01-09    0.546
2014-01-10    0.538
2014-01-11    0.506
2014-01-12    0.544
2014-01-13    0.587
2014-01-14    0.543
2014-01-15    0.544
2014-01-16    0.524
2014-01-17    0.520
2014-01-18    0.537
2014-01-19    0.577
2014-01-20    0.557
2014-01-21    0.575
2014-01-22    0.580
2014-01-23    0.547
2014-01-24    0.630
2014-01-25    0.597
2014-01-26    0.504
2014-01-27    0.480
2014-01-28    0.510
2014-01-29    0.473
2014-01-30    0.449
              ...  
2014-03-02    0.794
2014-03-03    0.823
2014-03-04    0.904
2014-03-05    0.903
2014-03-06    1.026
2014-03-07    0.837
2014-03-08    0.729
2014-03-09    0.720
2014-03-10    0.809
2014-03-11    0.741
2014-03-12    0.881
2014-03-13    0.854
2014-03-14    0.907
2014-03-15    0.716
2014-03-16    0.731
2014-03-17    0.897
2014-03-18    0.883
2014-03-19    0.821
2014-03-20 

In [71]:
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.8)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error

evaluate_arima_model(item1.iloc[:150,:]['Price'], (1,1,1))

/home/cchopade/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0.22890351173357082

In [78]:
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in tqdm(p_values):
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    
p = [1,2,4,6,8,10]
q = [1,2]
d = [1,2]

evaluate_models(item1.iloc[:365,:]['Price'],p,d,q)

  0%|          | 0/6 [00:00<?, ?it/s]

ARIMA(1, 1, 1) MSE=0.015


 17%|█▋        | 1/6 [00:45<03:47, 45.52s/it]

ARIMA(1, 1, 2) MSE=0.015
ARIMA(2, 1, 1) MSE=0.015


 33%|███▎      | 2/6 [01:25<02:51, 42.95s/it]

...

ARIMA(6, 1, 1) MSE=0.015
ARIMA(8, 1, 1) MSE=0.016


 83%|████████▎ | 5/6 [12:52<02:34, 154.56s/it]

ARIMA(10, 1, 1) MSE=0.018
ARIMA(10, 1, 2) MSE=0.016
ARIMA(10, 2, 1) MSE=0.009


100%|██████████| 6/6 [31:43<00:00, 317.32s/it]

Best ARIMA(10, 2, 1) MSE=0.009


In [42]:
import statsmodels.formula.api as sm

In [43]:
df.head()

,Item_ID,Datetime,ID,Category_3,Category_2,Category_1,Price,Number_Of_Sales,day,week,month,year
0,31375,2015-05-27,31375_20150527,0,5.0,235,0.882,133,27,22,5,2015
1,31375,2015-05-28,31375_20150528,0,5.0,235,0.787,113,28,22,5,2015
2,31375,2015-05-29,31375_20150529,0,5.0,235,0.942,59,29,22,5,2015
3,31375,2015-05-30,31375_20150530,0,5.0,235,1.049,56,30,22,5,2015
4,31375,2015-05-31,31375_20150531,0,5.0,235,1.381,37,31,22,5,2015


In [ ]:
FE_ols = sm.ols(formula='np.log(Price) ~ Datetime + Item_ID + C(Category_1) + C(Category_3)',data=df).fit()

In [44]:
from regressions.regressions import RDataFrame

In [45]:
dt = RDataFrame(df)

In [46]:
dt.xtset(i='Item_ID', t='Datetime')

In [57]:
dt.xtreg('np.log(Price) ~ day + week + month + year  - 1', 'fe', 'robust')

                            OLS Regression Results                            
Dep. Variable:          np.log(Price)   R-squared:                       0.583
Model:                            OLS   Adj. R-squared:                  0.583
Method:                 Least Squares   F-statistic:                 1.682e+10
Date:                Sun, 03 Jun 2018   Prob (F-statistic):               0.00
Time:                        05:25:33   Log-Likelihood:            -5.6548e+05
No. Observations:              881876   AIC:                         1.131e+06
Df Residuals:                  880344   BIC:                         1.131e+06
Df Model:                           4                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
day           -0.0014   4.86e-05    -28.263      0.0

<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x7f0e525b8da0>>